<a href="https://colab.research.google.com/github/raz0208/Agritech-Pest-Prediction/blob/main/Agritech_Pest_Prediction_Time_Series_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Time Series Analysis For Agritech Pest Prediction

## Model Implementation:
- ARIMA
- SARIMA
- ARIMAX
- SARIMAX

## **ARIMA**

In [1]:
# import required libaraies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load datasets
Merged_Dataset_df = pd.read_csv('/content/Final_Merged_Dataset_Cleaned.csv')
FeatureExtracted_df = pd.read_csv('/content/FeatureExtracted_dataset.csv')

In [4]:
# Display basic info for  datasets Final_Merged_Dataset_Cleaned
print(Merged_Dataset_df.info(), '\n')
print(Merged_Dataset_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Date                 153 non-null    object 
 1   Time                 153 non-null    object 
 2   Number of Insects    153 non-null    float64
 3   New Catches          153 non-null    float64
 4   Event                153 non-null    int64  
 5   Location             153 non-null    object 
 6   Average Temperature  153 non-null    float64
 7   Temp_low             153 non-null    float64
 8   Temp_high            153 non-null    float64
 9   Average Humidity     153 non-null    float64
 10  Day Avg_temp         153 non-null    float64
 11  Day Min_temp         153 non-null    float64
 12  Day Max_temp         153 non-null    float64
 13  Day Avg_Humidity     153 non-null    float64
 14  Temp_change          153 non-null    float64
dtypes: float64(11), int64(1), object(3)
memo